# Dyno Control Panel

This notebook provides a Python interface for controlling the Arduino dynamometer and generating torque-speed curves.

## Requirements
```bash
pip install pyserial matplotlib ipywidgets
```

In [ ]:
import serial
import serial.tools.list_ports
import time
import re
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from collections import defaultdict
import threading

## 1. Connect to Arduino

In [ ]:
# List available COM ports
ports = serial.tools.list_ports.comports()
print("Available ports:")
for port in ports:
    print(f"  {port.device}: {port.description}")

In [ ]:
# Connect to the Arduino
# Change COM port as needed (e.g., 'COM3' on Windows, '/dev/ttyUSB0' on Linux)
SERIAL_PORT = 'COM3'  # <-- CHANGE THIS
BAUD_RATE = 115200

ser = None

def connect():
    global ser
    try:
        ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
        time.sleep(2)  # Wait for Arduino to reset
        # Clear any startup messages
        while ser.in_waiting:
            print(ser.readline().decode('utf-8', errors='ignore').strip())
        print(f"Connected to {SERIAL_PORT}")
        return True
    except Exception as e:
        print(f"Connection failed: {e}")
        return False

def disconnect():
    global ser
    if ser and ser.is_open:
        ser.close()
        print("Disconnected")

def send_command(cmd):
    """Send a command to the Arduino and return the response."""
    if not ser or not ser.is_open:
        print("Not connected!")
        return []
    
    ser.write((cmd + '\n').encode())
    time.sleep(0.1)
    
    lines = []
    while ser.in_waiting:
        line = ser.readline().decode('utf-8', errors='ignore').strip()
        if line:
            lines.append(line)
    return lines

connect()

## 2. Basic Commands

In [ ]:
# Test connection with help command
response = send_command('help')
for line in response:
    print(line)

In [ ]:
# Tare (zero) the load cell before testing
response = send_command('tare')
for line in response:
    print(line)

In [ ]:
# Return brake to home position
response = send_command('brakeHome')
for line in response:
    print(line)

## 3. Run Automated Test

In [ ]:
def run_test(target_rpm, timeout=120):
    """
    Run an automated torque test at the specified RPM.
    
    Args:
        target_rpm: Target motor speed in RPM
        timeout: Maximum test duration in seconds
    
    Returns:
        dict with 'rpm', 'torque', 'brake_pos' lists and 'max_torque', 'stall_torque' values
    """
    if not ser or not ser.is_open:
        print("Not connected!")
        return None
    
    # Clear buffer
    while ser.in_waiting:
        ser.readline()
    
    # Start test
    ser.write(f'runTest {target_rpm}\n'.encode())
    
    data = {
        'rpm': [],
        'torque': [],
        'brake_pos': [],
        'max_torque': 0,
        'stall_torque': 0,
        'target_rpm': target_rpm,
        'stall_rpm': 0
    }
    
    start_time = time.time()
    test_complete = False
    
    print(f"Running test at {target_rpm} RPM...")
    
    while not test_complete and (time.time() - start_time) < timeout:
        if ser.in_waiting:
            line = ser.readline().decode('utf-8', errors='ignore').strip()
            
            # Parse DATA lines: DATA:rpm,torque,brake_pos
            if line.startswith('DATA:'):
                try:
                    parts = line[5:].split(',')
                    if len(parts) >= 3:
                        rpm = float(parts[0])
                        torque = float(parts[1])
                        brake_pos = int(parts[2])
                        data['rpm'].append(rpm)
                        data['torque'].append(torque)
                        data['brake_pos'].append(brake_pos)
                        print(f"  RPM: {rpm:.1f}, Torque: {torque:.4f} Nm, Brake: {brake_pos}")
                except ValueError:
                    pass
            
            # Parse RESULT line
            elif line.startswith('RESULT:'):
                # Parse: RESULT:maxTorque=X,stallTorque=Y,targetRPM=Z,stallRPM=W
                match = re.search(r'maxTorque=([\d.]+),stallTorque=([\d.]+),targetRPM=([\d.]+),stallRPM=([\d.]+)', line)
                if match:
                    data['max_torque'] = float(match.group(1))
                    data['stall_torque'] = float(match.group(2))
                    data['target_rpm'] = float(match.group(3))
                    data['stall_rpm'] = float(match.group(4))
            
            elif 'Test Complete' in line or 'STALL DETECTED' in line:
                test_complete = True
                print(line)
            
            elif line and not line.startswith('DATA'):
                print(line)
        
        time.sleep(0.01)
    
    if not test_complete:
        print("Test timed out!")
        send_command('abortTest')
    
    return data

In [ ]:
# Run a single test at 500 RPM
test_data = run_test(500)

if test_data:
    print(f"\nResults:")
    print(f"  Max torque: {test_data['max_torque']:.4f} Nm")
    print(f"  Stall torque: {test_data['stall_torque']:.4f} Nm")
    print(f"  Data points: {len(test_data['rpm'])}")

## 4. Generate Torque-Speed Curve

In [ ]:
def generate_torque_curve(rpm_values, delay_between_tests=5):
    """
    Run tests at multiple RPM values to generate a complete torque-speed curve.
    
    Args:
        rpm_values: List of RPM values to test
        delay_between_tests: Seconds to wait between tests
    
    Returns:
        dict with 'rpm' and 'max_torque' lists
    """
    results = {
        'rpm': [],
        'max_torque': [],
        'stall_torque': []
    }
    
    for i, rpm in enumerate(rpm_values):
        print(f"\n=== Test {i+1}/{len(rpm_values)}: {rpm} RPM ===")
        
        # Tare before each test
        send_command('tare')
        time.sleep(1)
        
        # Run test
        data = run_test(rpm)
        
        if data and data['max_torque'] > 0:
            results['rpm'].append(rpm)
            results['max_torque'].append(data['max_torque'])
            results['stall_torque'].append(data['stall_torque'])
        
        # Wait before next test
        if i < len(rpm_values) - 1:
            print(f"Waiting {delay_between_tests}s before next test...")
            time.sleep(delay_between_tests)
    
    return results

In [ ]:
# Generate torque-speed curve at multiple RPM values
rpm_test_points = [200, 400, 600, 800, 1000, 1200, 1500]

curve_data = generate_torque_curve(rpm_test_points)

In [ ]:
# Plot the torque-speed curve
plt.figure(figsize=(10, 6))
plt.plot(curve_data['rpm'], curve_data['max_torque'], 'bo-', linewidth=2, markersize=8, label='Max Torque')
plt.xlabel('Speed (RPM)', fontsize=12)
plt.ylabel('Torque (Nm)', fontsize=12)
plt.title('Motor Torque-Speed Curve', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('torque_curve.png', dpi=150)
plt.show()

print("\nCurve saved to torque_curve.png")

## 5. Interactive Control Panel

In [ ]:
# Create interactive widgets
output = widgets.Output()

rpm_slider = widgets.IntSlider(
    value=500,
    min=100,
    max=2000,
    step=100,
    description='Target RPM:',
    style={'description_width': 'initial'}
)

def on_run_test(btn):
    with output:
        clear_output()
        data = run_test(rpm_slider.value)
        if data and len(data['torque']) > 0:
            plt.figure(figsize=(8, 4))
            plt.plot(data['brake_pos'], data['torque'], 'g-')
            plt.xlabel('Brake Position (steps)')
            plt.ylabel('Torque (Nm)')
            plt.title(f'Test at {rpm_slider.value} RPM')
            plt.grid(True, alpha=0.3)
            plt.show()

def on_stop(btn):
    with output:
        clear_output()
        response = send_command('stop')
        for line in response:
            print(line)

def on_tare(btn):
    with output:
        clear_output()
        response = send_command('tare')
        for line in response:
            print(line)

def on_home(btn):
    with output:
        clear_output()
        response = send_command('brakeHome')
        for line in response:
            print(line)

run_btn = widgets.Button(description='Run Test', button_style='success')
stop_btn = widgets.Button(description='STOP', button_style='danger')
tare_btn = widgets.Button(description='Tare', button_style='info')
home_btn = widgets.Button(description='Brake Home', button_style='warning')

run_btn.on_click(on_run_test)
stop_btn.on_click(on_stop)
tare_btn.on_click(on_tare)
home_btn.on_click(on_home)

controls = widgets.HBox([run_btn, stop_btn, tare_btn, home_btn])
display(widgets.VBox([rpm_slider, controls, output]))

## 6. Cleanup

In [ ]:
# Disconnect when done
disconnect()